# Zadanie 1

Adrian Fabisiewicz <br> <br>
Nr indeksu: 328935

## Cel ćwiczenia
Zadanie polegało na wyznaczeniu położenia danej gwiazdy w układzie współrzędnych horyzontalnych, dla dwóch miejsc na Ziemi. Pierwsze miejsce znajduje się niedaleko Warszawy, jego przyjętą szerokością geograficzną było 52 stopnie, a długością - 21 stopni. Drugim rozpatrywanym miejscem był równik, miejsce o tej samej długości geograficznej, co pierwszy punkt. Obliczenia zostały wykonane dla całej doby, w godzinnych interwałach. Pierwsze wyliczone położenie gwiazdy miało miejsce 1 lipca 2023 roku o godzinie 02:00, a ostatnie - 2 lipca 2023 roku o godzinie 01:00. Wszystkie wyliczone położenia zwizualizowano na wykresach w sprawozdaniu.  

## Dane do zadania

Rozpatrywaną gwiazdą była gwiazda nr 168 z RA FK5, której współrzędne w układzie równikowym ekwinokcjalnym, na epokę 2023.5, wynosiły:
<li>α: 4<sup>h</sup> 37<sup>m</sup> 16.316<sup>s</sup></li>
<li>δ: 16° 33<sup>m</sup> 16.570<sup>s</sup></li>

## Kolejność wykonywania ćwiczeń

### Uwzględnienie strefy czasowej

Na początku należało zauważyć strefy czasowe punktów, biorąc pod uwagę, że znajdują się one w strefie UTC+2. W wyniku tego, w obliczeniach został uwzględniony okres od 1 lipca 2023, od godziny 2<sup>00</sup> do 2 lipca 2023, godziny 2<sup>00</sup> czasu UTC. Dzięki temu zabiegowi uzyskano położenia gwiazdy dla całej doby 1 lipca, od godziny 0 do 24, już w czasie UTC+2.

<br />

### Obliczenie lokalnego czasu gwiazdowego

Kolejnym krokiem było obliczenie lokalnego czasu gwiazdowego dla wybranych miejsc. Aby to zrobić, najpierw należało zapisać interesującą nas datę, używając kalendarza juliańskiego. Zastosowanie daty juliańskiej w obliczeniach astronomicznych między innymi pozwoliło na uniknięcie problemu przy zmianie kalendarza z juliańskiego na gregoriański czy zapewniło możliwość dokładnego obliczania odstępu w czasie od wydarzeń, które miały miejsce dawno w przeszłości. Do konwersji użyto uproszczoną wersję algorytmu, dostarczoną wraz z zadaniem, która pozwala na wykonanie poprawnych obliczeń w przedziale od 1 Marca 1900 do 28 lutego 2100. Wykorzystano przy tym poniższą funkcję <i>julday<i>.

In [ ]:
def julday(y,m,d,h):
    '''
    Simplified Julian Date generator, valid only between
    1 March 1900 to 28 February 2100
    '''
    if m <= 2:
        y = y - 1
        m = m + 12
    
    jd = np.floor(365.25*(y+4716))+np.floor(30.6001*(m+1))+d+h/24-1537.5
    return jd

Następnie, na podstawie otrzymanej daty juliańskiej, można było obliczyć czas średni gwiazdowy Greenwich (GMST). Aby to zrobić, skorzystałem z dostarczonej funkcji <i>GMST</i>, do której jako argument wstawiliśmy datę juliańską. Funkcja zwróciła czas średni gwiazdowy Greenwich, w godzinach.

In [ ]:
def GMST(jd):
    '''
    calculation of Greenwich Mean Sidereal Time - GMST in hours
    ----------
    jd : TYPE
        julian date
    '''
    T = (jd - 2451545) / 36525
    Tu = jd - 2451545
    g = 280.46061837 + 360.98564736629*(jd - 2451545.0) + 0.000387933*T**2-T**3/38710000
    g = (g%360) / 15
    return g

Aby teraz obliczyć lokalny czas gwiazdowy (LST), do uzyskanego czasu GMST, dodałem długość geograficzną miejsc obserwacji, wynoszącą 21 stopni, podzieloną przez 15, uzyskując LST wyrażony w godzinach.

In [ ]:
lst = gmst + warsaw[1]/15  #warsaw = [52, 21]

<br />

### Obliczenie kąta godzinnego

Mając lokalny czas gwiazdowy oraz daną wcześniej rektascenzję gwiazdy, mogłem przejść do obliczenia kąta godzinnego. Wykorzystałem do tego równanie: 

S = α<sup>∗</sup>+ t<sup>∗</sup>, <br/>
gdzie <br/> S - czas gwiazdowy miejscowy, 
<br/>α<sup>∗</sup> - rektascensja,<br/>
t<sup>∗</sup> - kąt godzinny

Po przekształceniu otrzymujemy wzór na kąt godzinny: <br/>
 t<sup>∗</sup> = S - α<sup>∗</sup>


 A więc:

In [ ]:
t_warsaw = lst_warsaw - alfa_h

<br />

### Rozwiązanie trójkąta sferycznego

Kolejnym krokiem zbliżającym do celu zadania, było rozwiązanie trójkąta sferycznego. Aby móc obliczyć wartości wysokości oraz azymutów, należało wyrazić na jego podstawie związku między układem współrzędnych horyzontalnych a układem
współrzędnych równikowych godzinnych. Po przekształceniach uzyskaliśmy wzór na wartość <i>h</i>:  $ \\ \sin h = \sin \phi \sin \delta + \cos \phi \cos \delta \cos t$

Azymut obliczyłem ze wzoru:
$\\ \tg Az = \frac{-\cos \delta \sin t}{\cos \phi \sin \delta - \sin \phi \cos \delta \cos t}$

Pozwoliło mi to na utworzenie funkcji, obliczających azymut oraz wysokość, na podstawie danej deklinacji, szerokości geograficznej oraz kątu godzinnego.

In [ ]:
# function to calculate height
def calculate_h(delta_r, fi_r, t_r):
    h = math.asin(math.sin(delta_r)*math.sin(fi_r) + math.cos(delta_r)*math.cos(fi_r)*math.cos(t_r))
    return h

# function to calculate azimuth
def calculate_a(delta_r, fi_r, t_r):
    a = -math.cos(delta_r)*math.sin(t_r)
    b = math.sin(delta_r)*math.cos(fi_r) - math.cos(delta_r)*math.sin(fi_r)*math.cos(t_r)
    az = math.atan2(a, b)
    return az

<br />

### Obliczenie wysokości oraz azymutu gwiazdy - transformacja do współrzędnych równikowych horyzontalnych



In [ ]:
# calculate h for warsaw (in degrees)
h_warsaw_r = calculate_h(delta_r, warsaw_r, t_warsaw_r)

# calculate azimuth for warsaw (in degrees)
a_warsaw_r = calculate_a(delta_r, warsaw_r, t_warsaw_r)

# calculate azimuth for equator (in degrees)
a_equator_r = calculate_a(delta_r, equator_r, t_equator_r)

# calculate h for equator (in degrees)
h_equator_r = calculate_h(delta_r, equator_r, t_equator_r)

<br />

## Wykonanie wizualizacji i przedstawienie wyników

<b><h2 style="text-align:center">Warszawa</h></b>

<div class="grid" style="display: grid;
            grid-template-columns: 1fr 1fr;
            grid-gap: 10px; ">
    <div>
        <img src="wykres1warszawa.png" alt="Obraz 1">
    </div>
    <div>
        <img src="wykres2warszawa.png" alt="Obraz 2">
    </div>
    <div>
        <img src="wykres4warszawa.png" alt="Obraz 3">
    </div>
    <div>
        <img src="wykres5warszawa.png" alt="Obraz 4">
    </div>
</div>

## Wnioski

<li>Gwiazda FK5 168 w miejscu w pobliżu Warszawy o współrzędnych φ = 52◦ oraz λ = 21◦ wzeszła 1 lipca 2023 r. krótko po godzinie 1<sup>00</sup> oraz zaszła około godziny 16<sup>00</sup></li>
<li>Najwyżej gwiazda znajdowała się krótko przed godziną 9<sup>00</sup>. Wysokość wynosiła wtedy blisko 55 stopni.</li>
<li>Gwiazda nie elonguje.</li>
<li>Gwiazda przechodzi przez I wertykał.</li>

<hr>
<b><h2 style="text-align:center">Równik</h></b>

<div class="grid" style="display: grid;
            grid-template-columns: 1fr 1fr;
            grid-gap: 10px; ">
    <div>
        <img src="wykres1rownik.png" alt="Obraz 1">
    </div>
    <div>
        <img src="wykres2rownik.png" alt="Obraz 2">
    </div>
    <div>
        <img src="wykres3rownik.png" alt="Obraz 3">
    </div>
    <div>
        <img src="wykres4rownik.png" alt="Obraz 4">
    </div>
</div>

## Wnioski

<li>Gwiazda FK5 168 w miejscu na równiku o współrzędnych φ = 0◦ oraz λ = 21◦ wzeszła 1 lipca 2023 r. około 2<sup>40</sup> oraz zaszła około godziny 14<sup>40</sup></li>
<li>Najwyżej gwiazda znajdowała się krótko przed godziną 9<sup>00</sup>. Wysokość wynosiła wtedy blisko 72 stopnie.</li>
<li>Gwiazda nie elonguje.</li>
<li>Gwiazda nie przechodzi przez I wertykał.</li>
<li>Gwiazda przez całą dobę była widoczna jedynie po północnej stronie nieba</li>